In [ ]:
# conda install -c anaconda keras

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Force TF to use only the CPU

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [ ]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('level1.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:
#         #tokenize each word
        w = nltk.word_tokenize(pattern)
#         print(w)
        words.extend(w)
#         #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
# classes
words
# # documents

In [ ]:
# ls

In [ ]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
# print(words)
words = sorted(list(set(words)))
# # sort classes        
classes = sorted(list(set(classes)))
# print(classes)
# # documents = combination between patterns and intents
print (len(documents), "documents")
# # classes = intents
print (len(classes), "classes", classes)
# # words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))


# # create our training data
training = []
# # create an empty array for our output
output_empty = [0] * len(classes)
# # training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    print(pattern_words)
#     # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    print(pattern_words)
    for w in words:
#         print(w,1 if w in pattern_words else 0)
        bag.append(1) if w in pattern_words else bag.append(0)
      
        
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

# print(bag)
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
print(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

len(train_x[0])

In [ ]:
len(train_y[0])

In [ ]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))                                                     
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('level1.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [ ]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def getOptions(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = i['options']
            break
    return result

def getRightKey(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = i['right_key']
            break
    return result

def getWrongKey(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = i['wrong_key']
            break
    return result

def getAnswer(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = i['answer']
            break
    return result

def getRightReward(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = i['right_reward']
            break
    return result

def getWrongReward(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = i['wrong_reward']
            break
    return result

answer_list = []
rewrd_list = []
user_answer_list = []

def getTotalReward(intents_json):
    total_reward = []
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        total_reward.append(i['right_reward'])
    return sum(total_reward)

def chatbot_response(msg,user_answer,reward):
    #user_answer_list.append(user_answer)
    try:
        ints = predict_class(msg, model)
        response = getResponse(ints, intents)
        options = getOptions(ints, intents)
        right_key = getRightKey(ints, intents)
        wrong_key = getWrongKey(ints, intents)
        right_reward = getRightReward(ints, intents)
        wrong_reward = getWrongReward(ints, intents)
        answer = getAnswer(ints, intents)
        
        #answer_list.append(answer)
        rewrd_list.append(int(reward))
    except IndexError:
        print ('Response Not Found!')   
    if msg == "no":
        byefn("Bye for Now")     
    elif msg == "bye":
        res = 'Pass'
        if sum(rewrd_list)<70:
            res = 'Fail'
        byefn('Total Rewards:'+str(sum(rewrd_list))+'/'+str(getTotalReward(intents))+'\nResult: '+res)
    else:
        try:
            button(response,options,right_key,wrong_key,right_reward,wrong_reward,answer)
        except IndexError:
            print ('Response Not Found!')
            res = 'Pass'
            if sum(rewrd_list)<70:
                res = 'Fail'
            byefn('Total Rewards:'+str(sum(rewrd_list))+'/'+str(getTotalReward(intents))+'\nResult: '+res)

In [ ]:
from tkinter import Tk,Frame,Canvas,Label,Button,BOTTOM

root = Tk()
root.config(bg='gray')
root.geometry("1000x500")
####parts of screen
#titlebar
    
f0=Frame(root,bg='gray',width=385, height=460, relief='raised', borderwidth=1)
f0.pack()
Label(f0,text='Test Your Skills',bg='gray',fg='black',font=('Helvetica 30 bold')).pack()

#maingame
c1=Canvas(root,bg='gray')
c1.pack()
#productthanks
f1=Frame(root,bg='gray')
f1.pack()
Label(f1,text='Welcome Guest',bg='gray',fg='black',font=('Arial 10 bold')).pack()
####frames
f2=Frame(c1,bg='gray')
f2.grid(row=0,column=0,stick='nsew')
f3=Frame(c1,bg='gray')
f3.grid(row=0,column=0,stick='nsew')
f4=Frame(c1,bg='gray')
f4.grid(row=0,column=0,stick='nsew')
f5=Frame(c1,bg='gray')
f5.grid(row=0,column=0,stick='nsew')

###welcome0
wel=Label(f2,text='Test Begin',fg='gray',bg='black',font=('Helvetica 30 bold'),pady=55)
wel.pack()


def start_ques():
    button("Are you ready to begin",["Yes","No"],"python","no",answer="Yes")
    rewrd_list = [0]
Button(f1,text='START',bd=0,bg='green',font=('Helvetica 30 bold'),fg='black',relief='flat',activebackground='gray',activeforeground='green',command=lambda: start_ques()).pack(side=BOTTOM)
    
def byefn(rew):
    f5.tkraise()
    nooo=Label(f5,pady=65,text=rew,fg='black',bg='gray',font=('Helvetica 30 bold'))
    nooo.pack()

def button(res,ops,right_key,wrong_key="no",right_reward=0,wrong_reward=0,answer="python"):
    f3=Frame(c1,bg='gray')
    f3.grid(row=0,column=0,stick='nsew')
    q1=Label(f3,text=res,fg='black',bg='gray',font=('Helvetica 14 bold'))
    q1.pack()
    ch = 1
    for op in ops:
        if op == answer:
            key = right_key
            reward = right_reward
        else:
            key = wrong_key
            reward = wrong_reward
        button_options(f3,str(ch)+'. '+op,key,reward)
        ch += 1
        print(key)
    
def button_options(fr_no,op,key,reward):
    Button(fr_no,text=op,bd=0,fg='green',font=('Helvetica 15 bold'),bg='black',relief='flat',activebackground='gray',activeforeground='green',command=lambda:chatbot_response(key,op,reward)).pack()
        
root.title('GoFree Passport')
root.mainloop()